## set running directory

In [ ]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [ ]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing-deep")
try_mkdir(TEST_DATA_PATH)

In [ ]:
VISUALIZE = True
PLAY_RESULT = False
CLEARANCE = 1e-3
TOOL_RPY = (-np.pi/2,0,0)
TIMEOUT_MOTION = 5
MAX_TIME = 100


ROBOT_TYPE = RobotType.indy7gripper

if ROBOT_TYPE in [RobotType.indy7, RobotType.indy7gripper]:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.14)
    HOME_POSE = (0,0,0,0,0,0)
    GRIP_DEPTH = 0.05
    
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.112)
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
    GRIP_DEPTH = 0.03
    
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")

## init combined robot config

In [ ]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [ ]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


In [ ]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [ ]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [ ]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [ ]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

## ui

In [ ]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

# Object Classes

In [ ]:
from pkg.planning.pddlstream.scene_building import *

### sampling 1-obj

In [ ]:
Nmax_obj = 1
f_option = "obj_1"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01


gscene.clear_link("base_link")
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = gscene.get_gtem_args()
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
    time.sleep(0.2)

### sampling 1-obj-3obstacle

In [ ]:
Nmax_obj = 1
f_option = "obj_1_obs3"
Nmax_obs = 3

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


gscene.clear_link("base_link")
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)
    
    for i_obs in range(Nmax_obs):
        colliding = True
        while colliding:
            pole = random.choice([Pole, Bar])(gscene, "obs"+str(i_obs))
            colliding = (wp.is_overlapped_with(pole.geometry, margin=OBS_MARGIN) 
                         or gp.is_overlapped_with(pole.geometry, margin=OBS_MARGIN))

    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = gscene.get_gtem_args()
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
    time.sleep(0.2)

### sampling 1-obj-5obstacle

In [ ]:
Nmax_obj = 1
f_option = "obj_1_obs5"
Nmax_obs = 5

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


gscene.clear_link("base_link")
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)
    
    for i_obs in range(Nmax_obs):
        colliding = True
        while colliding:
            pole = random.choice([Pole, Bar])(gscene, "obs"+str(i_obs))
            colliding = (pole.is_overlapped_with(wp.geometry, margin=OBS_MARGIN) 
                         or pole.is_overlapped_with(gp.geometry, margin=OBS_MARGIN))

    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = gscene.get_gtem_args()
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
    time.sleep(0.2)

In [ ]:

## add floor, ceiling
floor = Floor(gscene, "floor")
gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                   dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                   color=floor.geometry.color, display=True, collision=True, fixed=True)
# ceiling = Ceiling(gscene, "ceiling")

## set workplane
wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

gp_colliding = True
while gp_colliding:
    ## set goalplane
    gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
    gp_colliding = wp.is_overlapped_with(gp.geometry)
pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

for i_obs in range(Nmax_obs):
    colliding = True
    while colliding:
        pole = random.choice([Pole, Bar])(gscene, "obs"+str(i_obs))
        colliding = (pole.is_overlapped_with(wp.geometry, margin=OBS_MARGIN) 
                     or pole.is_overlapped_with(gp.geometry, margin=OBS_MARGIN))

## add object
obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

gtem_args = gscene.get_gtem_args()
obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
            for obj in obj_list}


### sampling 3-obj

In [ ]:
Nmax_obj = 3
f_option = "obj_3"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01


gscene.clear_link("base_link")
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = gscene.get_gtem_args()
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
    time.sleep(0.2)

## sample with pole

In [ ]:
Nmax_obj = 3
f_option = "obj_3_pole"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01


gscene.clear_link("base_link")
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

#     pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])
    
    for _ in range(10):
        Tgp = gp.geometry.get_tf(crob.home_pose)
        Pp = (np.matmul(Tgp[:3,:3], np.transpose([[-0.3,0,0]])*(1+np.random.rand()))+Tgp[:3,3:]).flatten()

        Ppcyl = cart2cyl(*Pp)
        pole = Pole(gscene, "pole", RTH=[np.maximum(0.3,Ppcyl[0]-0.1), Ppcyl[1], 0])
        colliding = pole.is_overlapped_with(wp.geometry) or pole.is_overlapped_with(gp.geometry)
        if not colliding:
            break


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = gscene.get_gtem_args()
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
    time.sleep(0.2)

## sample difficult

In [ ]:
WorkPlane.DIM_MIN = (0.6, 0.8, 0.05)
WorkPlane.DIM_MAX = (0.6, 0.8, 0.05)


Pole.DIM_MIN = (0.2, 0.2, 4)
Pole.DIM_MAX = (0.2, 0.2, 4)

PlaneObject.GTYPE = GEOTYPE.CYLINDER
PlaneObject.DIM_MAX = (0.05,0.05,0.2)
PlaneObject.DIM_MIN = (0.05,0.05,0.2)


In [ ]:
Nmax_obj = 3
Nplane_obj  = 1
f_option = "obj_3_hard"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01

gscene.clear_link("base_link")
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set goalplane
    gp = WorkPlane(gscene, "gp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set workplane
        wp = SideBox(gscene, "wp", floor_height=gp.RTH[2]+gp.RTH[2]/2)
        gp_colliding = gp.is_overlapped_with(wp.geometry)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

#     pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])

    Tgp = gp.geometry.get_tf(crob.home_pose)
    Pp = (np.matmul(Tgp[:3,:3], np.transpose([[-0.3,0,0]])*(1+np.random.rand()))+Tgp[:3,3:]).flatten()

#     Ppcyl = cart2cyl(*Pp)
#     pole = Pole(gscene, "pole", RTH=[np.maximum(0.3,Ppcyl[0]-0.1), Ppcyl[1], 0])
    
    colliding = True
    while colliding:
        pole = Pole(gscene, "pole")
        colliding = (pole.is_overlapped_with(wp.geometry, margin=OBS_MARGIN) 
                     or pole.is_overlapped_with(gp.geometry, margin=OBS_MARGIN))


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)
    disperse_objects(gscene, PlaneObjectBig, "obs", Nplane_obj, workplane_on=gp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = gscene.get_gtem_args()
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    # sort object data by x-pose in workspace
    T_wp = wp.geometry.get_tf(crob.home_pose)
    onames_original = sorted([obj_.name for obj_ in obj_list])
    gargs_original = {garg["name"]: garg for garg in gtem_args if garg["name"] in onames_original}
    onames_sorted = sorted(onames_original, 
                           key=lambda x: -np.matmul(SE3_inv(T_wp), gscene.NAME_DICT[x].get_tf(crob.home_pose))[0,3])
    obj_args_bak = deepcopy(obj_args)
    for oname_from, oname_to in zip(onames_sorted, onames_original):
        for garg in gtem_args:
            if garg["name"] == oname_from:
                garg["name"] = oname_to+"_xx"
            if garg["parent"] == oname_from:
                garg["parent"] = oname_to+"_xx"
        obj_args[oname_to] = deepcopy(obj_args_bak[oname_from])
        obj_args[oname_to]["gname"] = obj_args_bak[oname_to]["gname"]
    for garg in gtem_args:
        if garg["name"].endswith("_xx"):
            garg["name"] = garg["name"][:-3]
        if garg["parent"] is not None and garg["parent"].endswith("_xx"):
            garg["parent"] = garg["name"][:-3]
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
    time.sleep(0.2)